# Experiment 1
I try to process a specific text here.

In [40]:
text = '__团队成员#1__先生是公司创始人,也是中国最有影响力的商界领袖之一。1982年,__团队成员#1__先生于华南理工大学毕业,进入TCL的前身-TTK家庭电器有限公司。1985年,他担任新成立的TCL通讯设备公司总经理,创立了TCL品牌。2003年,__团队成员#1__担任TCL集团股份有限公司董事长兼CEO,随后TCL集团整体上市。在他的领导下,2004年TCL一举收购了法国汤姆逊全球彩电业务与阿尔卡特全球手机业务。目前TCL集团已经成为拥有6万名员工,业务遍及全球80多个国家和地区。2013年,TCL集团营业总收入超过855亿元,液晶电视全球销量1766万台,实际产量全球第三,品牌销售全球第三;TCL手机全球销量5520万台,行业排名全球第五。2012年__团队成员#1__被新华网评为“最具社会责任感企业家”;2011年荣获《中国企业家》“最具影响力的25位企业领袖”终身成就奖;2009年被评为“CCTV中国经济年度人物十年商业领袖”;2008年获改革开放30年经济人物称号;2004年被评为Fortune杂志“亚洲年度经济人物”、TIME杂志和CNN全球最具影响力的25名商界人士,同年法国总统希拉克向__团队成员#1__先生颁发了法国国家荣誉勋章。__团队成员#1__是中共第十六大代表,第十届、第十一届、第十二届全国人大代表。__团队成员#1__担任的社会职务包括:中国电子视像行业协会会长;中国国际商会副会长;全国工商联执行委员、广东省工商联(总商会)副主席。'

In [41]:
print(text)

__团队成员#1__先生是公司创始人,也是中国最有影响力的商界领袖之一。1982年,__团队成员#1__先生于华南理工大学毕业,进入TCL的前身-TTK家庭电器有限公司。1985年,他担任新成立的TCL通讯设备公司总经理,创立了TCL品牌。2003年,__团队成员#1__担任TCL集团股份有限公司董事长兼CEO,随后TCL集团整体上市。在他的领导下,2004年TCL一举收购了法国汤姆逊全球彩电业务与阿尔卡特全球手机业务。目前TCL集团已经成为拥有6万名员工,业务遍及全球80多个国家和地区。2013年,TCL集团营业总收入超过855亿元,液晶电视全球销量1766万台,实际产量全球第三,品牌销售全球第三;TCL手机全球销量5520万台,行业排名全球第五。2012年__团队成员#1__被新华网评为“最具社会责任感企业家”;2011年荣获《中国企业家》“最具影响力的25位企业领袖”终身成就奖;2009年被评为“CCTV中国经济年度人物十年商业领袖”;2008年获改革开放30年经济人物称号;2004年被评为Fortune杂志“亚洲年度经济人物”、TIME杂志和CNN全球最具影响力的25名商界人士,同年法国总统希拉克向__团队成员#1__先生颁发了法国国家荣誉勋章。__团队成员#1__是中共第十六大代表,第十届、第十一届、第十二届全国人大代表。__团队成员#1__担任的社会职务包括:中国电子视像行业协会会长;中国国际商会副会长;全国工商联执行委员、广东省工商联(总商会)副主席。


In [42]:
import jieba
import re
import chardet
from gensim.models import KeyedVectors

In [43]:
stopwords = {}
fstop = open('data/stopwords_cn.txt', 'r', encoding='utf-8', errors='ignore')
for w in fstop:
    stopwords[w.strip()] = w.strip()

fstop.close()
stopwords[' '] = ' '

In [44]:
# re.sub(r"[\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——！，;:。？、~@#￥%……&*（）]+", " ", data)

In [87]:
text = re.sub(r"[\s+\.\!\/_,$%^*()?;；:【】+\"\']+|[+——！，;:。？、~@#￥%……&*（）]+", " ", text)
text = text.lower()
print(text)

 团队成员 1 先生是公司创始人 也是中国最有影响力的商界领袖之一 1982年 团队成员 1 先生于华南理工大学毕业 进入tcl的前身-ttk家庭电器有限公司 1985年 他担任新成立的tcl通讯设备公司总经理 创立了tcl品牌 2003年 团队成员 1 担任tcl集团股份有限公司董事长兼ceo 随后tcl集团整体上市 在他的领导下 2004年tcl一举收购了法国汤姆逊全球彩电业务与阿尔卡特全球手机业务 目前tcl集团已经成为拥有6万名员工 业务遍及全球80多个国家和地区 2013年 tcl集团营业总收入超过855亿元 液晶电视全球销量1766万台 实际产量全球第三 品牌销售全球第三 tcl手机全球销量5520万台 行业排名全球第五 2012年 团队成员 1 被新华网评为“最具社会责任感企业家” 2011年荣获《中国企业家》“最具影响力的25位企业领袖”终身成就奖 2009年被评为“cctv中国经济年度人物十年商业领袖” 2008年获改革开放30年经济人物称号 2004年被评为fortune杂志“亚洲年度经济人物” time杂志和cnn全球最具影响力的25名商界人士 同年法国总统希拉克向 团队成员 1 先生颁发了法国国家荣誉勋章 团队成员 1 是中共第十六大代表 第十届 第十一届 第十二届全国人大代表 团队成员 1 担任的社会职务包括 中国电子视像行业协会会长 中国国际商会副会长 全国工商联执行委员 广东省工商联 总商会 副主席 


In [96]:
words = jieba.cut(text, cut_all=False)
seglist = list(words)
print(seglist)

[' ', '团队', '成员', ' ', '1', ' ', '先生', '是', '公司', '创始人', ' ', '也', '是', '中国', '最有', '影响力', '的', '商界', '领袖', '之一', ' ', '1982', '年', ' ', '团队', '成员', ' ', '1', ' ', '先生', '于', '华南理工大学', '毕业', ' ', '进入', 'tcl', '的', '前身', '-', 'ttk', '家庭', '电器', '有限公司', ' ', '1985', '年', ' ', '他', '担任', '新', '成立', '的', 'tcl', '通讯设备', '公司', '总经理', ' ', '创立', '了', 'tcl', '品牌', ' ', '2003', '年', ' ', '团队', '成员', ' ', '1', ' ', '担任', 'tcl', '集团股份', '有限公司', '董事长', '兼', 'ceo', ' ', '随后', 'tcl', '集团', '整体', '上市', ' ', '在', '他', '的', '领导', '下', ' ', '2004', '年', 'tcl', '一举', '收购', '了', '法国', '汤姆逊', '全球', '彩电', '业务', '与', '阿尔卡特', '全球', '手机', '业务', ' ', '目前', 'tcl', '集团', '已经', '成为', '拥有', '6', '万名', '员工', ' ', '业务', '遍及', '全球', '80', '多个', '国家', '和', '地区', ' ', '2013', '年', ' ', 'tcl', '集团', '营业', '总收入', '超过', '855', '亿元', ' ', '液晶电视', '全球', '销量', '1766', '万台', ' ', '实际', '产量', '全球', '第三', ' ', '品牌', '销售', '全球', '第三', ' ', 'tcl', '手机', '全球', '销量', '5520', '万台', ' ', '行业', '排名', '全球', '第五', ' ', '2012', '年', ' ', 

In [97]:
segListSanitized = []

for word in seglist:
    if word not in stopwords:
        segListSanitized.append(word)
print(segListSanitized)

['团队', '成员', '1', '先生', '公司', '创始人', '中国', '最有', '影响力', '商界', '领袖', '之一', '1982', '年', '团队', '成员', '1', '先生', '华南理工大学', '毕业', '进入', 'tcl', '前身', '-', 'ttk', '家庭', '电器', '有限公司', '1985', '年', '担任', '新', '成立', 'tcl', '通讯设备', '公司', '总经理', '创立', 'tcl', '品牌', '2003', '年', '团队', '成员', '1', '担任', 'tcl', '集团股份', '有限公司', '董事长', '兼', 'ceo', 'tcl', '集团', '整体', '上市', '领导', '2004', '年', 'tcl', '一举', '收购', '法国', '汤姆逊', '全球', '彩电', '业务', '阿尔卡特', '全球', '手机', '业务', '目前', 'tcl', '集团', '已经', '成为', '拥有', '6', '万名', '员工', '业务', '遍及', '全球', '80', '多个', '国家', '地区', '2013', '年', 'tcl', '集团', '营业', '总收入', '超过', '855', '亿元', '液晶电视', '全球', '销量', '1766', '万台', '实际', '产量', '全球', '第三', '品牌', '销售', '全球', '第三', 'tcl', '手机', '全球', '销量', '5520', '万台', '行业', '排名', '全球', '第五', '2012', '年', '团队', '成员', '1', '新华网', '评为', '“', '最具', '社会', '责任感', '企业家', '”', '2011', '年', '荣获', '《', '中国', '企业家', '》', '“', '最具', '影响力', '25', '位', '企业', '领袖', '”', '终身', '成就奖', '2009', '年', '评为', '“', 'cctv', '中国', '经济', '年度人物', '十年', '商业', '领袖',

In [98]:
print(len(seglist))
print(len(segListSanitized))

323
239


In [99]:
def getNgrams(wordList, n):
    output = set()
    for i in range (len(wordList) - n + 1):
        n_gram_temp = "".join(wordList[i:i+n])
        output.add(n_gram_temp)
    return output

In [100]:
output1 = getNgrams(segListSanitized, 1)
print(output1)

{'工商联', '1', '团队', '包括', '年度', '总经理', '年', '汤姆逊', '产量', '主席', '集团股份', '视像', '855', '副会长', 'tcl', '同年', 'ceo', '先生', '5520', 'fortune', '影响力', '阿尔卡特', '一举', 'cctv', '上市', '经济', '国家', '总商会', '成就奖', '希拉克', '领导', '《', '电子', '全球', '2012', '代表', '委员', '企业', '销售', '企业家', '社会', '家庭', '1985', '整体', '总收入', '国际商会', '十年', 'time', '目前', '亿元', '大', '勋章', '法国', '》', '商界', '副', '排名', '广东省', '第十六', '第十届', '通讯设备', '有限公司', '人大代表', '评为', '兼', '集团', '人物', '杂志', '进入', '2011', '获', '拥有', '30', '中共', '全国', '成为', '已经', '成立', '荣誉', '第十二届', '第十一届', '毕业', '创立', '彩电', '新', '称号', '终身', '领袖', '万台', '第五', '担任', '1982', '董事长', '中国', '商业', '电器', '名', '手机', '全国工商联', '25', '80', '改革开放', '位', '协会会长', '2004', '品牌', '2003', '遍及', '2013', '华南理工大学', '总统', '6', '2009', '行业', '员工', '前身', '责任感', '颁发', '人士', '之一', '创始人', '最具', '最有', '年度人物', '亚洲', 'cnn', '公司', '-', '荣获', '第三', '多个', '收购', '2008', '成员', '1766', '销量', '实际', 'ttk', '业务', '地区', '职务', '执行', '”', '液晶电视', '超过', '营业', '万名', '新华网', '“'}


In [101]:
output2 = getNgrams(segListSanitized, 2)
print(output2)

{'颁发法国', 'tcl一举', '”终身', '产量全球', '年担任', '员工业务', '勋章团队', '企业家”', '彩电业务', '全球第五', '6万名', '“cctv', '企业家》', '评为“', '已经成为', '年评为', '评为fortune', '有限公司1985', '第十六大', '职务包括', '商界人士', '万台行业', '位企业', '企业领袖', '全球手机', '25名', '第三品牌', '有限公司董事长', '人物称号', '阿尔卡特全球', '总商会副', '十年商业', '商业领袖', '1新华网', '30年', '领袖之一', '5520万台', '前身-', '中国最有', '-ttk', '担任tcl', '销售全球', '领袖”', '最具影响力', '工商联总商会', '手机业务', '中国经济', '希拉克团队', '先生颁发', '称号2004', '先生华南理工大学', '”2008', '《中国', '新成立', '行业排名', '品牌2003', '名商界', '公司创始人', '收购法国', '1982年', '副主席', '创始人中国', 'tcl集团', '执行委员', '大代表', '一举收购', '获改革开放', 'tcl品牌', 'cctv中国', '经济年度人物', '代表第十届', '全国人大代表', 'tcl集团股份', '集团整体', '之一1982', '成就奖2009', '电器有限公司', '委员广东省', '创立tcl', '董事长兼', '法国汤姆逊', '目前tcl', '全球最具', '万名员工', '年度经济', '家庭电器', '商界领袖', '年经济', '1766万台', '集团已经', '责任感企业家', '广东省工商联', '年度人物十年', '业务目前', '销量5520', '影响力25', '毕业进入', '协会会长中国', '销量1766', '第十一届第十二届', '年tcl', '拥有6', '品牌销售', 'fortune杂志', '影响力商界', '2013年', '排名全球', '“最具', '亿元液晶电视', '兼ceo', '副会长全国工商联', '80多个', '万台实际', '第十届第十一届', '杂志“', 'ceo

In [102]:
output3 = getNgrams(segListSanitized, 3)
print(output3)

{'tcl集团股份有限公司', '已经成为拥有', '整体上市领导', '液晶电视全球销量', '年评为“', '企业家”2011', '2013年tcl', '评为“最具', '企业家》“', '中国经济年度人物', '”2011年', '毕业进入tcl', '行业排名全球', '中国最有影响力', '一举收购法国', '集团整体上市', '国家荣誉勋章', '大代表第十届', '30年经济', '业务遍及全球', '商界人士同年', '多个国家地区', 'tcl品牌2003', '总经理创立tcl', '地区2013年', '先生公司创始人', '企业领袖”', '《中国企业家', '“最具影响力', '2003年团队', '人物称号2004', '年tcl一举', '25名商界', '1先生颁发', '同年法国总统', 'tcl前身-', '年获改革开放', '1担任社会', '“最具社会', '全球第三tcl', '排名全球第五', 'tcl集团整体', '超过855亿元', '杂志“亚洲', '新成立tcl', '领袖之一1982', '全球销量1766', '第十届第十一届第十二届', 'tcl集团已经', '领导2004年', '第十六大代表', '2009年评为', '手机业务目前', '年荣获《', '成员1中共', '2011年荣获', '总商会副主席', '全球第五2012', '第三tcl手机', '名商界人士', '商界领袖之一', '年经济人物', '目前tcl集团', '1先生华南理工大学', '终身成就奖2009', '领袖”终身', '年评为fortune', '副会长全国工商联执行', '担任社会职务', '成为拥有6', '电子视像行业', '》“最具', '2004年评为', '成员1先生', '1中共第十六', '1985年担任', '协会会长中国国际商会', '6万名员工', '集团股份有限公司董事长', '2004年tcl', '有限公司1985年', '汤姆逊全球彩电', '经济人物称号', '全球销量5520', '称号2004年', '视像行业协会会长', '通讯设备公司总经理', 'fortune杂志“', '第五2012年', '收购法国汤姆逊', '实际产量全球', '彩电业务阿尔卡特', '成员1新华网',

In [103]:
final = output1 | output2 | output3
print(final)

{'工商联', 'tcl集团股份有限公司', '已经成为拥有', '液晶电视全球销量', '年评为“', '”终身', '企业家”2011', '产量全球', '年担任', '企业家》“', '企业家”', '包括', '中国最有影响力', '6万名', '一举收购法国', '总经理', '集团整体上市', '国家荣誉勋章', '评为fortune', '已经成为', '30年经济', '汤姆逊', '多个国家地区', '第十六大', 'tcl品牌2003', '主席', '商界人士', '总经理创立tcl', '先生公司创始人', '集团股份', '企业领袖”', '企业领袖', '全球手机', '《中国企业家', '“最具影响力', '25名商界', 'tcl', '1先生颁发', '同年法国总统', 'tcl前身-', '年获改革开放', '1担任社会', '先生', '总商会副', '5520', '“最具社会', '全球第三tcl', 'tcl集团整体', 'fortune', '影响力', '新成立tcl', '一举', 'cctv', '1新华网', '领袖之一1982', '领袖之一', '5520万台', '上市', '中国最有', '国家', '-ttk', '销售全球', '第十六大代表', '手机业务目前', '总商会', '成就奖', '工商联总商会', '中国经济', '成员1中共', '2011年荣获', '先生华南理工大学', '全球第五2012', '《中国', '2012', '代表', '商界领袖之一', '委员', '年经济人物', '收购法国', '创始人中国', 'tcl集团', '销售', '终身成就奖2009', '大代表', '一举收购', '担任社会职务', 'tcl品牌', 'cctv中国', '经济年度人物', '代表第十届', '全国人大代表', '整体', '委员广东省', '成就奖2009', '电器有限公司', '1中共第十六', '创立tcl', '全球最具', '协会会长中国国际商会', 'time', '6万名员工', '集团股份有限公司董事长', '目前', '家庭电器', '商界领袖', '年经济', '1766万台', '大', '勋章', '法国', '》', '集团已经', '责任感企业

In [90]:
model = KeyedVectors.load('./test_10.bin')

In [92]:
v = model['已经成为']
model.most_similar('tcl集团')

[('tcl多媒体', 0.8246365189552307),
 ('华星光电', 0.8007965087890625),
 ('创维集团', 0.7881876826286316),
 ('创维数字', 0.7685035467147827),
 ('海信电器', 0.7675223350524902),
 ('兆驰股份', 0.7568047046661377),
 ('创维数码', 0.7451937198638916),
 ('胜宏科技', 0.7427794933319092),
 ('大族激光', 0.7411966323852539),
 ('万和电气', 0.7410907745361328)]

In [129]:
import numpy as np
def calculate_cosine_similarity(a, b):
    vector_a = np.mat(a)
    vector_b = np.mat(b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

In [144]:
def generateEmbeddings(name):
    words = jieba.cut(name, cut_all=False)
    word_list = list(words)
    v = np.zeros((200))
    for word in word_list:
        if word in model.vocab:
            v += model[word]
    
    v /= len(v)
    return v

In [145]:
import pandas as pd
path0 = 'data/chinese_university_list.csv'

In [146]:
school_df = pd.read_csv(path0, header=None, delimiter=",", skiprows=4, names=["rank", "name", "code", "department", "city", "level", "notes"])
print(school_df.head())

  rank    name          code department city level notes
0    1    北京大学  4.111010e+09        教育部  北京市    本科   NaN
1    2  中国人民大学  4.111010e+09        教育部  北京市    本科   NaN
2    3    清华大学  4.111010e+09        教育部  北京市    本科   NaN
3    4  北京交通大学  4.111010e+09        教育部  北京市    本科   NaN
4    5  北京工业大学  4.111010e+09        北京市  北京市    本科   NaN


In [147]:
school_df = pd.DataFrame(school_df, columns=['rank', 'name'])
print(school_df.head())
print(school_df.shape)

  rank    name
0    1    北京大学
1    2  中国人民大学
2    3    清华大学
3    4  北京交通大学
4    5  北京工业大学
(2718, 2)


In [154]:
for term in final:
    if term in model.vocab:
        term_emb = model[term]
        school_candidate = dict()
        for index, row in school_df.iterrows():
            name = row['name']
            if isinstance(name, float):
                continue
            if name in model.vocab:
                name_emb = model[name]
            else:
                name_emb = generateEmbeddings(name)
            
            sim = calculate_cosine_similarity(term_emb, name_emb)
            if (sim > 0.9):
                school_candidate[row['name']] = sim
        if len(school_candidate) == 0:
            continue
        school_candidate = sorted(school_candidate.items(), key=lambda item:item[1], reverse=True)
        print(f'university entity found: {term}->{school_candidate[0][0]}')

c:\users\梁育诚\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


university entity found: 华南理工大学->华南理工大学


In [104]:
import pandas as pd
path4 = 'data/company_list_ch.csv'

In [105]:
company_ch_df = pd.read_csv(path4, header=None, delimiter=",", skiprows=1, names=['rank', 'Name', 'Location', 'Income'])
print(company_ch_df.head())

   rank             Name Location        Income
0  名次\n           企业名称\n     地区\n  营业收入\n(亿元)\n
1   1\n   中国石油化工集团有限公司\n     北京\n    27427.80\n
2   2\n  中国石油天然气集团有限公司\n     北京\n    25994.17\n
3   3\n       国家电网有限公司\n     北京\n    25602.54\n
4   4\n     中国建筑股份有限公司\n     北京\n    11993.25\n


In [158]:
import string
for term in final:
    if len(term) <= 1:
        continue
    if term in model.vocab:
        term_emb = model[term]
        company_ch_candidate = dict()
        for index, row in company_ch_df.iterrows():
            name = row['Name']
            name = name.lower()
            if str.find(name, term) == -1:
                continue
#             words = jieba.cut(name, cut_all=False)
#             name_list = list(words)
#             li1 = getNgrams(name_list, 1)
#             li2 = getNgrams(name_list, 2)
#             li = li1 | li2
            
            if name in model.vocab:
                name_emb = model[name]
            else:
                name_emb = generateEmbeddings(name)
            sim = calculate_cosine_similarity(term_emb, name_emb)
            if (sim > 0.8):
                company_ch_candidate[name] = sim
        if len(company_ch_candidate) == 0:
            continue
        company_ch_candidate = sorted(company_ch_candidate.items(), key=lambda item:item[1], reverse=True)
        print(f'company entity found: {term}->{company_ch_candidate[0][0]}')

company entity found: 集团股份->利华益集团股份有限公司

company entity found: tcl->tcl集团股份有限公司

company entity found: 国家->国家电力投资集团有限公司

company entity found: tcl集团->tcl集团股份有限公司

company entity found: 广东省->广东省交通集团有限公司

company entity found: 集团->青建集团

company entity found: 中国->中国国际技术智力合作有限公司

company entity found: 公司->北京外企服务集团有限责任公司

company entity found: 中国电子->中国电子信息产业集团有限公司

company entity found: 电子->中国电子信息产业集团有限公司

company entity found: 企业->企业名称

company entity found: 有限公司->中国石油化工集团有限公司

company entity found: 集团股份有限公司->新凤鸣集团股份有限公司

